### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 3.5.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run single_alpha_decay_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    
    alpha = params[0]
    beta = params[1]
    decay = params[2]

    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], alpha)
        # then update value
        EVs[t] = update_value(Probs[t]) 
        
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(t)
        for x in options:
            Probs[x] = Probs[x] + decay * (0.5 - Probs[x])
            EVs[x] =  update_value(Probs[x]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
# check simulation 
#sim_data(15,[.2, 2, .2])

### New function to sim data for multiple subjects

In [9]:
def sim_mult_subject_data(n, tn, params):
    data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha', 'Beta', 'Decay'])
    for x in range(1, n+1):
        sdt = sim_data(tn, params)
        sdt = pd.DataFrame.from_dict(sdt)
        sdt['Subject'] = [x for t in range(0,tn*3)]
        sdt['Alpha'] = params[0]
        sdt['Beta'] = params[1]
        sdt['Decay'] = params[2]
        data = pd.concat([data,sdt])
    return data

In [10]:
# multisubject simulation
#data = sim_mult_subject_data(10, 15, [.2, 2, .4])
#data

### Create param space

In [15]:
# create parameter space
alphas = [round(x * 0.1,1) for x in range(0,11)]
betas = [x * 2 for x in range(0,11)]
decays = [round(x * 0.1,1) for x in range(0,11)]
combos = list(itertools.product(alphas, betas, decays))
#for r in combos: print(r)

### Simulate data

In [16]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha', 'Beta', 'Decay'])
for p in range(len(combos)):
    dt = sim_mult_subject_data(1, 15, combos[p])
    dt['Subject'] = dt['Subject'] + p*10
    data = pd.concat([data, dt])
    
data.to_csv(path_or_buf = '../../output/simulation/sim_1alpha_decay_model_data.csv', index = False)

### New function to fit model to multiple subjects

In [17]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'alpha', 'beta', 'decay'])
    a = pd.unique(data['Subject'])
    print('Number of Subs: ', len(a))
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==(sub)]
        dt = df.to_dict()
        params = model_fit(dt)
        #print(params)
        line = {'Subject': sub, 'alpha': params.x[0], 'beta': params.x[1], 'decay': params.x[2]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [18]:
precover = model_fit_mult_subject(data)
precover.to_csv(path_or_buf = '../../output/simulation/sim_1alpha_decay_model_fits.csv', index = False)

Number of Subs:  1331
Subject:  1
Subject:  2


KeyError: 0

In [ ]:
print(precover)